In [1]:
# if error < tol: # MSE in yhat and y is less than some threshold
#     break

import numpy as np
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

boston = load_boston()
X = boston.data
y = boston.target
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

intercept = np.ones((X_train.shape[0],1))
X_train = np.concatenate((intercept, X_train), axis=1) # axis 1 is COLUMN

In [2]:
intercept = np.ones((X_test.shape[0],1))
X_test = np.concatenate((intercept, X_test), axis=1)

In [3]:
from numpy.linalg import inv # inverse matrix

def closed_form(X, y):
    return inv(X.T @ X) @ X.T @ y

theta = closed_form(X_train, y_train)
#print(theta)
yhat = X_test @ theta # multiply testing dataset with weights (yhat = predicted y)

print(yhat.shape)
print(y_test.shape)
print(yhat[56:60])
print(y_test[56:60])

(152,)
(152,)
[25.25986846 39.76111867 28.44843866 32.33508408]
[25.3 21.9 28.  31.1]


In [4]:
mse = ((y_test - yhat)**2).sum()/X_test.shape[0]
mse

22.633570222300364